In [1]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:15<00:00,  1.89s/it]


In [55]:
import sys
del sys.modules["split_llm.glm6b.wrapped_layer"]

In [56]:
from split_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, FeedForward_GLM_Wrapped, copy_attantion, copy_feedforward

In [57]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)
feedforward_wrapped = FeedForward_GLM_Wrapped(4096, 32, 0)
copy_attantion(transformer_layer, attn_wrapped)
copy_feedforward(transformer_layer, None, feedforward_wrapped)

In [58]:
import torch
from split_llm.glm6b.utils import generate_attention_mask, generate_position_ids

In [59]:
xs = torch.rand([10, 1, 4096]).to("cuda:1")
transformer_layer.to("cuda:1")
attn_wrapped.to("cuda:1")
feedforward_wrapped.to("cuda:1")
position_ids = generate_position_ids(10, 10).to("cuda:1")
attention_mask = generate_attention_mask(10, 10).to("cuda:1")

In [63]:
output_raw = transformer_layer(xs, position_ids, attention_mask, 0)[0]
print(output_raw.shape)
print(output_raw)

torch.Size([10, 1, 4096])
tensor([[[ -5.0948, -11.7365,  11.4381,  ...,  -4.1215,  -9.7167, -11.9493]],

        [[  0.9475,  17.7879,   8.4281,  ..., -12.0736,  -2.7919,  14.0786]],

        [[ -7.0398,  -9.1274,   9.5206,  ...,   1.8684,  -2.1848,  -1.1104]],

        ...,

        [[ -6.6490,  -4.1403,  -2.4016,  ..., -11.8345, -11.4994,  -1.1972]],

        [[  7.4000,   7.5975,   1.6621,  ...,  -6.7609,  -0.8396,  -4.7317]],

        [[ -6.2023,   1.8287,   7.2340,  ..., -14.0230,  -8.8436,  -5.2539]]],
       device='cuda:1')


In [64]:
# Compute the output by wrapped layers
xs = transformer_layer.input_layernorm(xs)
attn_out = attn_wrapped(xs, position_ids)
attn_out += (2 * 28) ** 0.5 * xs
output_wrapped = feedforward_wrapped(attn_out)


In [62]:
print(output_wrapped.shape)
print(output_wrapped)

torch.Size([10, 1, 4096])
tensor([[[ -5.0572, -12.3509,  11.0530,  ...,  -3.7901,  -9.0527, -11.8540]],

        [[  1.5295,  17.7705,   8.5445,  ..., -13.2526,  -3.0623,  13.9441]],

        [[ -6.7168,  -9.9631,   9.4791,  ...,   1.2421,  -2.4624,  -1.1486]],

        ...,

        [[ -6.1324,  -4.0719,  -3.2749,  ..., -11.7572, -10.7988,  -0.8559]],

        [[  7.9249,   7.5566,   1.1843,  ...,  -6.7077,  -1.2108,  -4.6363]],

        [[ -6.5769,   2.6066,   6.5732,  ..., -14.8396,  -9.0565,  -5.1362]]],
       device='cuda:1', grad_fn=<AddBackward0>)


In [37]:
attention_mask

tensor([[[[False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False]]]],
       device='cuda:1')